# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys_adams import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
my_weather = "output_data/my_weather_data.csv"

vacation_df = pd.read_csv(my_weather)

vacation_df


In [ ]:
vacation_df.info()

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
gmaps.configure(api_key=g_key)

In [ ]:
locations = vacation_df.loc[:, ["Lattitude", "Longitude"]]
fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(locations, weights=vacation_df['Humidity'],max_intensity=20, point_radius=5)
fig.add_layer(heatmap_layer)
fig

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
#Data is in Celcius. Converted to personal preferences in Celcius.
mask= (vacation_df["Max Temperature"] < 32) & (vacation_df["Max Temperature"] > 18) & (vacation_df["Wind Speed"] < 10) & (vacation_df.Cloudiness == 0)
my_cities= vacation_df.loc[mask]
my_cities

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_names = []
ratings = []
cities = []
countries = []
lats = []
longs = []


for _, row in my_cities.iterrows():
    print(row.City)
    
    target_coordinates = f"{row.Lattitude}, {row.Longitude}"
    target_search = ""

    target_radius = 5000
    target_type = "lodging"

    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params)

    places_data = response.json()

    hotel = places_data["results"][0]["name"]
    rating = places_data["results"][0]["rating"]
    
    hotel_names.append(hotel)
    ratings.append(rating)
    cities.append(row.City)
    countries.append(row.Country)
    lats.append(row.Lattitude)
    longs.append(row.Longitude)


In [ ]:

hotel_df = pd.DataFrame()
hotel_df['City'] = cities
hotel_df['Country'] = countries
hotel_df['Lattitude'] = lats
hotel_df['Longitude'] = longs
hotel_df['Hotel'] = hotel_names
hotel_df['Rating'] = ratings

hotel_df

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df["Lattitude", "Longitude"]

In [ ]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(hotel, info_box_content=hotel_info)
heatmap_layer = gmaps.heatmap_layer(locations, weights=hotel_df['Rating'],max_intensity=20, point_radius=5)

fig = gmaps.figure()

# add layers
fig.add_layer(marker_layer)
fig.add_layer(heatmap_layer)

# Display figure
fig